# NSMC - NAVER Sentiment Movie Corpus

> gpt3 백본모델로 KoAlpaca를 활용하여 인컨텍스트 퓨삿러닝을 통해 네이버 영화 리뷰 데이터(NSMC - NAVER Sentiment Movie Corpus) 분류문제를 풀어보자.

참고로 `KoAlpaca` 는 한국어로 인스트럭션 데이터를 한국어 오픈소스 gpt3 모델인 polyglot-ko에 파인튜닝한 모델입니다.

In [4]:
# !pip install transformers
# !pip install accelerate

In [3]:
!mkdir -p data_in/KOR/naver_movie
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_train.txt \
              -O data_in/KOR/naver_movie/ratings_train.txt
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_test.txt \
              -O data_in/KOR/naver_movie/ratings_test.txt

--2023-04-02 21:25:32--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘data_in/KOR/naver_movie/ratings_train.txt’

data_in/KOR/naver_m 100%[===================>]  13.95M  --.-KB/s    in 0.1s    

2023-04-02 21:25:34 (97.0 MB/s) - ‘data_in/KOR/naver_movie/ratings_train.txt’ saved [14628807/14628807]

--2023-04-02 21:25:34--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/naver_movie/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.g

In [21]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer

import torch
import pandas as pd
import numpy as np

import re

import random
from random import sample

from tqdm import tqdm

In [22]:
# torch.cuda.is_available()
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [23]:
SEED_NUM = 1234
random.seed(SEED_NUM)

In [24]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/polyglot-ko-5.8b")

In [25]:
cls_model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot",
                                                 torch_dtype=torch.float16,
                                                 device_map='sequential',
                                                 low_cpu_mem_usage=True).cuda()

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.69s/it]


In [26]:
cls_model.config.max_length = 2048
cls_model.config.pad_token_id = 0

In [27]:
# 데이터 전처리 준비
DATA_IN_PATH = './data_in/KOR'
DATA_OUT_PATH = './data_out/KOR'

DATA_TRAIN_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_train.txt')
DATA_TEST_PATH = os.path.join(DATA_IN_PATH, 'naver_movie', 'ratings_test.txt')

train_data = pd.read_csv(DATA_TRAIN_PATH, header = 0, delimiter = '\t', quoting = 3)
train_data = train_data.dropna()

In [28]:
print('데이터 positive 라벨: ', '긍정')
print('데이터 negative 라벨: ', '부정')

데이터 positive 라벨:  긍정
데이터 negative 라벨:  부정


In [29]:
print('학습 예시 케이스 구조: ', '문장: 오늘 기분이 좋아\n감정: 긍정\n')

학습 예시 케이스 구조:  문장: 오늘 기분이 좋아
감정: 긍정



In [30]:
print('gpt3 최대 토큰 길이: ', cls_model.config.max_position_embeddings)

gpt3 최대 토큰 길이:  2048


In [31]:
sent_lens = [len(tokenizer(s).input_ids) for s in tqdm(train_data['document'])]

print('Few shot 케이스 토큰 평균 길이: ', np.mean(sent_lens))
print('Few shot 케이스 토큰 최대 길이: ', np.max(sent_lens))
print('Few shot 케이스 토큰 길이 표준편차: ',np.std(sent_lens))
print('Few shot 케이스 토큰 길이 80 퍼센타일: ',np.percentile(sent_lens, 80))

100%|████████████████████████████████████████████████████████████████████████████████████████| 149995/149995 [00:07<00:00, 21048.91it/s]


Few shot 케이스 토큰 평균 길이:  20.22912763758792
Few shot 케이스 토큰 최대 길이:  280
Few shot 케이스 토큰 길이 표준편차:  16.48828728915166
Few shot 케이스 토큰 길이 80 퍼센타일:  27.0


In [32]:
train_fewshot_data = []

for train_sent, train_label in tqdm(train_data[['document', 'label']].values):
    tokens = tokenizer(train_sent).input_ids

    if len(tokens) <= 25:
        train_fewshot_data.append((train_sent, train_label))

100%|████████████████████████████████████████████████████████████████████████████████████████| 149995/149995 [00:07<00:00, 20502.42it/s]


In [71]:
test_data = pd.read_csv(DATA_TEST_PATH, header=0, delimiter='\t', quoting=3)
test_data = test_data.dropna()
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [75]:
# 평가 데이터 수
sample_size = 500

# 평가에서 활용할 few-shot 예제를 묶음으로 저장
train_fewshot_samples = []

for _ in range(sample_size):
    # few-shot 예제를 10개씩 묶음
    fewshot_examples = sample(train_fewshot_data, 10)
    train_fewshot_samples.append(fewshot_examples)

if sample_size < len(test_data['id']):
    test_data = test_data.sample(sample_size, random_state=SEED_NUM)

In [76]:
train_fewshot_samples[1]

[('레전드.', 1),
 ('보고 있으면 어느새 영화에 빠져들어 있다. 마지막 장면을 생각하면 아직도 가슴이 찡하다.', 1),
 ('오즈의 영역을 넘어 마술의 경지에 이르는 허우.', 1),
 ("'스트레스 해소용 영화' 로써만 강추..", 0),
 ('볼만햇다 액션영화가 아닌데 아쉽다', 1),
 ('귀여운 기니피그가 나오는... 디즈니 영화. 디즈니 디즈니 디즈니', 0),
 ('햄릿! 돈은 있는데.. 정말 매너리즘에 빠진 최악의 중국 영화', 0),
 ('영화도 좋지만 책도 꼭 읽어보세요ㅎㅎ', 1),
 ('많은 생각이 떠돌게 되는 영화...', 1),
 ('개봉일을 잡았다는게 신기한 싸구려 영화.. 싼맛에 수입한거겠지?', 0)]

In [77]:
def build_prompt_text(sent):
    return "문장: " + sent + '\n감정:'

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", sent)
    return sent_clean

def generate_fewshot_example(data, with_label=True):
    example_text, example_label = data
    # 텍스트 전처리
    cleaned_example_text = clean_text(example_text)
    # Prompt 형식 구성
    fewshot_example_text = build_prompt_text(cleaned_example_text)
    # Label 추가
    if with_label:
      fewshot_example_text += ' 긍정' if example_label == 1 else ' 부정' + '\n'
    
    return fewshot_example_text

def predict_by_generation(prompt_text):
    # 토큰화 및 인덱싱
    tokens = tokenizer(prompt_text, return_tensors="pt")
    token_ids, attn_mask = tokens.input_ids.cuda(), tokens.attention_mask.cuda()
    # 텍스트 생성
    gen_tokens = cls_model.generate(input_ids=token_ids, attention_mask=attn_mask,
                                    max_new_tokens=1, pad_token_id=0)
    # 인덱스 복호화
    pred = tokenizer.batch_decode(gen_tokens[:, -1])[0].strip()
    
    return pred

In [74]:
test_data

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [78]:
real_labels = []
pred_tokens = []

total_len = len(test_data[['document','label']].values)

for i, row in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        prompt_text += generate_fewshot_example(ex)

    prompt_text += generate_fewshot_example(row, with_label=False)

    pred = predict_by_generation(prompt_text)

    pred_tokens.append(pred)
    real_labels.append('긍정' if row[1] == 1 else '부정')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:30<00:00, 16.16it/s]


In [79]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)

0.762


In [80]:
def build_prompt_text(sent):
    return '다음 문장은 긍정일까요 부정일까요?\n' + sent + '\n정답:'

real_labels = []
pred_tokens = []

total_len = len(test_data[['document','label']].values)

for i, row in tqdm(enumerate(test_data[['document','label']].values), total=total_len):
    prompt_text = ''

    for ex in train_fewshot_samples[i]:
        prompt_text += generate_fewshot_example(ex)

    prompt_text += generate_fewshot_example(row, with_label=False)

    pred = predict_by_generation(prompt_text)

    pred_tokens.append(pred)
    real_labels.append('긍정' if row[1] == 1 else '부정')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:39<00:00, 12.80it/s]


In [81]:
accuracy_match = [p == t for p, t in zip(pred_tokens, real_labels)]
accuracy = len([m for m in accuracy_match if m]) / len(real_labels)

print(accuracy)

0.744
